In [9]:
import os
import leafmap
import torch
from samgeo.hq_sam import SamGeo, show_image, download_file, overlay_images, tms_to_geotiff

In [2]:
coords = [33.67766, 73.13274]
m = leafmap.Map(center=coords, zoom=16)
m.add_basemap("SATELLITE") #"HYBRID"
m

Map(center=[33.67766, 73.13274], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

In [5]:
image = "17-02-2023-Reprojected.tif"
modelPath = "E:\Engg Softwares\GIS\SAM\sam_hq_vit_h.pth"
# m.layers[-1].visible = False  # turn off the basemap
m.add_raster(image, layer_name="Image")

In [6]:
sam_kwargs = {
    "points_per_side": 32,
    "pred_iou_thresh": 0.86,
    "stability_score_thresh": 0.92,
    "crop_n_layers": 1,
    "crop_n_points_downscale_factor": 2,
    "min_mask_region_area": 100,
}

In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
sam = SamGeo(
    checkpoint=modelPath,
    model_type='vit_h',
    device='cpu',
    sam_kwargs=None,
)

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
sam.generate(image, output="masks.tif", foreground=True, unique=True, batch=True)

In [ ]:
sam.show_masks(cmap="binary_r")

In [ ]:
sam.show_anns(axis="off", alpha=1, output="annotations.tif")

In [ ]:
leafmap.image_comparison(
    "satellite.tif",
    "annotations.tif",
    label1="Satellite Image",
    label2="Image Segmentation",
)

In [ ]:
m.add_raster("annotations.tif", alpha=0.5, layer_name="Masks")

In [ ]:
sam.tiff_to_vector("annotations.tif", "masks.gpkg")